# __From Narrow AI to Generative AI: Enhancing your DFIR Skills with AI__
__SANS-AI Cybersecurity Summit__

*by Jess Garcia (@j3ssgarcia) and Mario Perez*

## 🌙🚨 __The Midnight Intruder Case__ 🌙🚨

In the dead of night 🦉, an intruder gained unauthorized access to the company's network via a __compromised user account obtained through phishing__ 🎣. Their goal was to deploy ransomware 💻🔒, but to do so, they needed administrative privileges. After several hours of __brute-forcing__ 🔑💥, they succeeded in cracking an admin password, __escalating privileges__ 🚀. However, the __creation of an admin domain controller account 'sql001'__ 🖥️ triggered an alert 🚨, allowing the IT team to detect the breach just minutes before the ransomware could be deployed.

Can you trace the intruder's steps, identify the breached accounts and machines 🕵️‍♂️ to neutralize the threat before it's too late? ⏳

__For your referece, this is the Attackmap of the Threat Actor__

<div>
<img src="attackmap.jpeg" width="890"/>
</div>

### __FORENSIC EVIDENCE__ 

This section describes the evidence we will be examining in this case. Our focus will be on analyzing Windows EVTX files. 

__Windows EVTX files__ are log files utilized by the Windows operating system to document various events, including system, security, and application activities. These logs can capture events ranging from software installations to security incidents and system errors. The EVTX format is binary, which offers advantages in terms of storage efficiency and retrieval speed compared to plain text logs. __To analyze these files, we can use tools such as the Windows Event Viewer, or specialized parsers like EvtxECmd or Sabonis__.

#### __Sabonis__ 

Sabonis is a __Digital Forensics and Incident Response pivoting tool__ for parsing different kind of forensic artifacts realted to lateral movements like __EVTX, proxy and PCAP files__. These are the main features of the tool:
* 🔍 Extracts and merge lateral movements from more than 7 different EVTX files
* 🔍 Parses Squid proxy events
* 🔍 Extracts all lateral movements from PCAP files
* ⚡ Quick and low memory comsumption
* 📑 Loads different sources into a Neo4J database
* 🔍 Includes a Cypher Playbook to make investigations easy

You can find more information at: https://github.com/jupyterj0nes/sabonis

#### __Creating a Lateral Movement Dataset with Sabonis__

Sabonis is a powerful tool for __consolidating lateral movement event logs from your network into a single CSV dataset__ for analysis. It processes lateral movement events from EVTX files, including:

__C:/Windows/System32/winevt/Logs/__
* _Security.evtx_
* _Microsoft-Windows-SMBServer%4Security.evtx_
* _Microsoft-Windows-SmbClient%4Security.evtx_
* _Microsoft-Windows-TerminalServices-RDPClient%4Operational.evtx_
* _Microsoft-Windows-RemoteDesktopServices-RdpCoreTS%4Operational.evtx_

__Using Sabonis__

In [15]:
# Do not run this cell; it is just an example Sabonis' usage. It is meant to be run in the console. Pre-cooked evidence will be provided to you later.

# ./pivotfoot.sh source_folder_with_evtxs/ destination_folder/
# python3 sabonis.py parse evtx --source_artifact destination_folder/ --directory --csv_output sabonis_output.csv

https://github.com/jupyterj0nes/sabonis?tab=readme-ov-file#examples

__Pre-cooked Evidence__

In this lab, you'll be working with pre-cooked evidence generated by running Sabonis on various EVTX files collected from different machines on the network, including domain controllers, servers, and workstations.

You can find the consolidated file, __sabonis_output.csv__, in the /datasets folder within the same directory as this notebook: https://github.com/ds4n6/sans_narrow-ai_notebooks.

## __LAB-001:__ Anomaly Detection with ML for DFIR

With all the necessary information in hand, we can start the investigation 🕵️‍♂️.

__Our goal__ is to monitor the intruder's activities, evaluate the nature of the attack, and assess the extent of the damage caused. To efficiently achieve this, we will leverage advanced __Data Science and Machine Learning__ techniques. This approach will enable us to analyze patterns, detect anomalies, and gain insights that are crucial for a comprehensive understanding of the intrusion. Let’s get started!!

To carry out the analysis, we will use the __ds4n6_lib__ python library, also known as __CHRYSALIS__ (https://github.com/ds4n6/ds4n6_lib).

__CHRYSALIS__ is a Python library for __Digital Forensics and Incident Response (DFIR)__ that enables the application of __AI/ML analyses__ on critical forensic artifacts like EVTX files, Amcache, Autoruns, or Pslist. It can also work directly with the output from various forensic tools, including Plaso, Kape, Fstl, Kansa, Mactime, Volatility, or Sabonis, among others.

__To start the analysis__, we first need to load the Sabonis module from CHYRSALIS, which allows us to read and work with Sabonis datasets. Then, we’ll load the "_explore_" functionality, which we’ll use later to examine the dataset's content and conduct a preliminary analysis.

In [8]:
# IMPORT LIBRARIES
import ds4n6_lib.sabonis as d4sbns
from ds4n6_lib.tools import explore

#### __TASK 001-001:__ Read the Sabonis Dataset with CHRYSALIS

__Run the following cell__ to read the Sabonis dataset and displays it on the screen. The dataset is loaded as a __Pandas dataframe__ and stored in the variable '__sbnsdf__'.

__NOTE:__ A Pandas DataFrame is a python data structure used for storing datasets. It organizes data into rows and columns, making it straightforward to manipulate, analyze, and visualize the data. The Pandas library also offers a range of powerful functions to handle, clean, and analyze data within these DataFrames.

In [ ]:
path = './datasets/sabonis_output.csv'
sbnsdf = d4sbns.read_data(path, tool='sabonis')
sbnsdf

__Understanding the Dataset__

As previously mentioned, Sabonis parses data from various EVTX files to extract information related to lateral movements. Some of the key fields of interest include:
* __Timestamp__: The date and time of the event.
* __EventID__: The Windows event identifier.
* __LogonType__: The type of logon (Interactive, Network, Batch, Remote-Interactive...).
* __ComputerName__: The target machine of the lateral movement (destination machine).
* __TargetUserName__: The name of the user that logons on the target machine.
* __SourceComputer__: The machine from which the lateral movement originated (source machine).
* __SourceIP__: The IP address of the source machine.
* __evtxFileName__: The name of the EVTX file containing the event data.

Take some time to review the dataset and understand the data it contains. Use the down bar to move to the right or to the left. __As you can see, the dataset has more than 83k events!!!__

#### __TASK 001-002:__ Explore the Dataset

Before diving into more complex analysis, we'll begin with a __preliminary exploration__ to assess the dataset's scope, the types of data it contains, and key metrics such as the number of machines or users contained in the dataset. 

An __initial qualitative analysis__ is usually __quick__, aids in gaining a __clearer understanding__ of the dataset, and often __helps in identifying anomalies early on__.

To explore the dataset we are going to use the __CHRYSALIS__ function __'explore()'__ that we loaded at the beginning. This function takes an input DataFrame (df) and a column name (col) as arguments, and __prints the counts of unique values__ in that column of the dataset. 

__Run the following cell__ to analyze the users involved in lateral movements within the network. We will focus on the '_TargetUserName__' column of the dataset '__sbnsdf__' for this analysis.

In [ ]:
explore(sbnsdf,"TargetUserName_")

As you can see, the user '__administrator__' appears in 32,639 events as the target username, along with other users such as '__admin__', '__localroot__', or '__it-support__'.

__Now, use the CHRYSALIS' explore() funtion to answer the following questions...__

<span style="color:blue">**What is the most repeated event IDs in the dataset?** </span>

In [ ]:
# Put your code here and run the cell (TIP: check again the name of the columns of the dataset)


<span style="color:green">**ANSWER HERE**:</span>

<span style="color:blue">**What is the machine that receives the most traffic on the network?** </span>

In [ ]:
# Put your code here and run the cell


<span style="color:green">**ANSWER HERE**:</span>

#### __TASK 001-003:__ Machine Learning Analysis

__Now's when the magic happens! ⭐__

__CHRYSALIS__ is designed for analyzing diverse forensic artifacts by using __Artificial Intelligence and Machine Learning__ techniques. The library implements high-level functions to assist forensic experts in their investigations. When analyzing __Sabonis datasets__, CHRYSALIS organizes logon events into two categories: successful (__event ID 4624__) and failed (__event ID 4625__), grouping them by minutes. Whith these processed samples, CHRYSALIS trains Machine Learning models to __identify the most suspicious events on the network__.

__Run the following cell__ to apply ML techniques for detecting the most suspicious login activity on the network. For this analysis, we just need to run the CHRYSALIS' function __ml_logons_anomalies()__. This funtion receives three arguments:
* __df__ = the Sabonis dataset to analyze, in our case '__sbnsdf__'
* __model__ = the ML model used for anomaly detection. Here, we use the model __iForest (Isolation Forest)__, a simple yet effective ML method for identifying outliers in large datasets. 
* __top_n__ = the number of anomalies or outliers to display.

The funtion __returns__ a sorted list with the most anomalous samples of the dataset. Note that samples at the beginning of the list will be more anomalous than those toward the end. 

__TIP__: To analyze the results, focus on the columns "_EventID_4624__" and "_EventID_4625__". They show the __count of events 4624 and 4625__ where the user '_TargetUserName__' tried to access to the target machine '_LatMov__', within the minute specified in the column '_Timestamp__'.

In [ ]:
logon_anomalies_iforest = d4sbns.ml_logons_anomalies(sbnsdf, model='iforest', top_n=50)
logon_anomalies_iforest

<span style="color:blue">**Identify two possible users who may have been abused with brute force** </span>

<span style="color:green">**ANSWER HERE**:</span>

<span style="color:blue">**What day and time does the brute force campaign take place?** </span>

<span style="color:green">**ANSWER HERE**:</span>

<span style="color:blue">**Based on the results, Do you think the brute force campaign was successful?** </span>

<span style="color:green">**ANSWER HERE**:</span>

__Autoencoders for Anomaly Detection__

In CHRYSALIS, we can use either __iForest__ models or __Deep Learning Autoencoders__ for detecting anomalies in our datasets. To accomplish this, simple switch the model to '__simple_autoencoder__' when using the __ml_logons_anomalies()__ function. This allows you to easily switch between different anomaly detection techniques based on your specific needs and dataset characteristics.

Note that we can set the number of training epochs for the ML model using the '__train_epochs__' argument. We'll keep the number of epochs low to prevent overtraining the network, improving the capabilities of the model to detect anomalies.

In [ ]:
logon_anomalies_ae = d4sbns.ml_logons_anomalies(sbnsdf, model='simple_autoencoder', top_n=50, train_epochs=3)
logon_anomalies_ae

Analyzing the results, we note that __the Autoencoder also highlights some events indicative of brute force attacks__. However, the __iForest model demonstrated superior performance in detecting brute force__ attempts made by the user '__administrator__' on the target machine '__WRKSTN-100__'.

<span style="color:blue">**[Advanced] On the Autoencoder output, Do you see any signs of suspicious activity that the iForest model did not detect?** </span>

__TIP: Pay special attention to the information revealed by the name of the machines and the type of traffic they typically receive.__

<span style="color:green">**ANSWER HERE**:</span>

## __LAB-002:__ Graphs & ML for Lateral Movement Detection

We know that __the threat actor has gained access to multiple machines__ on the network and compromised two user accounts: '__administrator__' and '__sql001__'. In this lab, we'll conduct a more __in-depth analysis of the lateral movements__ within the network.

__Graph-based analysis__ is a highly effective method for examining network graphs and identifying suspicious lateral movements (_Graphs for DFIR Analysis. The Roadmap: http://www.ds4n6.io/blog/23050801.html_). However, in large-scale networks, this approach can become complicated due to the vast number of events occurring within the network. In these scenarios, the malicious activity of an attacker may be masked by the background noise generated by legitimate users, systems, or IT administrators.

__CHRYSALIS for Graph Analysis__

Since the release 0.8.1, the __ds4n6_lib__ library implements a module to __apply Machine Learning on Graphs__ to discover __stealth actors moving laterally through large networks__. With the module __mlgraph__, we can represent the logon activity of a network in a graph and __apply advanced Machine Learning models__ to detect suspicious lateral movements in the network.

To begin with the __lateral movement analysis__, we first need to load the __mlgraph__ module of CHRYSALIS. Next, we will reload the Sabonis dataset as we did in the previous lab session.

In [ ]:
import ds4n6_lib.mlgraph as d4mlg
import ds4n6_lib.sabonis as d4sbns

path = './datasets/sabonis_output.csv'
sbnsdf = d4sbns.read_data(path, tool='sabonis')
sbnsdf

#### __TASK 002-001:__ Lateral Movement Dataset

__Thinking in Graphs__

For thinking in graphs, we first need to represent our data in that format. So, the initial step is to convert the data into a graph representation. CHRYSALIS makes this easier with the function '__build_lm_dataset()__'. This function turns an event log dataset into a graph by creating two matrices: the feature matrix and the adjacency matrix. It then tie the user sessions based on the logon events timestamps. Finally, it generates a new dataset containing details about user activity sub-graphs, which represents the __users' lateral movements__ in the network. 

More details about '__build_lm_dataset()__' funtion at http://www.ds4n6.io/blog/23091501.html

__Run the following cell__ to tie the events logs and generate a __new dataset with the complete lateral movements__ of the users in the network. As argument, '__build_lm_dataset()__' receives the Sabonis dataset we loaded before.

In [ ]:
lm_dataset = d4mlg.build_lm_dataset(sbnsdf)
lm_dataset

As shown, __CHRYSALIS has identified 650 lateral movements__ from our dataset. Initially, we observe that these movements vary in length, with some being 2 jumps long and others extending to 3 jumps. The '__path__' column displays the sequence of movements made by a user within a specific time period.

#### __TASK 002-002:__ Machine Learning Analysis

After building the new dataset with the lateral movements of the network, let's analyze it for searching anomalies. For this purpose, CHRYSALIS provides the '__find_lm_anomalies()__' function. This function leverages an __Autoencoder based on Transformers__ to __identify the most suspicious lateral movements in the network__.

__find_lm_anomalies()__ function requires six parameters:
* __dataset__: The lateral movements dataset generated CHRYSALIS.
* __model__: The type of ML model for anomaly detection.
* __from_date__: The start date of the analysis.
* __to_date__: The end date of the analysis.
* __top_n__: The number of anomalies or outliers to display.
* __neo4j__: Indicates whether to generate a compatible Neo4j dataset to load the results into a Neo4j database for further analysis.

Further information here: http://www.ds4n6.io/blog/23091501.html

__Run the following cell__ to look for suspicious lateral movements. Initially, you will observe the Autoencoder training process. Afterward, the model will show a ranking of the most anomalous lateral movements in the network. 

__NOTE__: The '_Error_' parameter indicates the anomaly level, where 100% signifies a highly anomalous movement and 0% indicates minimal anomaly. Keep in mind that most machine learning models include a stochastic component, meaning that running the model multiple times can yield different outcomes. Nonetheless, if you run the model several times, you'll notice that the results tend to be quite similar.

In [ ]:
output = d4mlg.find_lm_anomalies(lm_dataset, model='transformer', from_date='2023-01-01', to_date='2024-01-17',top_n=30, neo4j=False)

<span style="color:blue">**Lists the lateral movements of the threat actor detected by the model (As reference, use the attack map provided at the beginning of the notebook)** </span>

__Note: Results may vary due to the stochastic nature of the model. We suggest running the model multiple times and reviewing the outcomes.__

<span style="color:green">**ANSWER HERE**:</span>

__Congratulations on completing the cybersecurity incident response challenge!!__ 🎉

You’ve successfully navigated through this real-world CTF scenario, demonstrating your skills and creativity in __solving complex problems with AI and Machine Learning__.

If you enjoyed this challenge and are eager to explore more content like this, we invite you to visit our website at https://www.ds4n6.io/. There, you'll find a variety of engaging and educational resources designed to further enhance your __Threat Detection & Response__ expertise.

Thank you for participating, and we look forward to seeing you in future challenges! 🚀